Stacked AutoEncoder

Import the Libraries Needed

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

Import the Datasets

In [2]:
#Column 0 is Movie ID, 1 is the title, 2 is the genre
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, engine='python', encoding='latin-1')
movies.head()

,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, engine='python', encoding='latin-1')
users.head(10)

,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


In [4]:
#Column 0 is UserID, 1 is MovieID, 2 is the Ratings, 3 is timestamps and will be removed
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, engine='python', encoding='latin-1')
ratings.head(10)

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


Preparing the train and test set

In [5]:
#This contains 80k reviews of the 100k so this is 80% of the data
training_set = np.array(pd.read_csv('ml-100k/u1.base', delimiter='\t'), dtype='int')

In [6]:
test_set = np.array(pd.read_csv('ml-100k/u1.test', delimiter='\t'), dtype='int')

Getting the number of users and movies

In [7]:
#Get the max number among both training and test set for the amount of users and amount of movies
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

Convert the data into an array with users in lines and movies in columns

In [8]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:, 1][data[:, 0] == id_users]
        id_ratings = data[:, 2][data[:, 0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data

In [9]:
training_set = convert(training_set)
test_set = convert(test_set)

Converting the data into Pytorch Tensors

In [10]:
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

Creating the architecture of the Neural Network (SAE = Stacked AutoEncoder)

In [16]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x     

In [17]:
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5) 

Training the SAE

In [18]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
  train_loss = 0
  s = 0.
  for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = input.clone()
    if torch.sum(target.data > 0) > 0:
      output = sae(input)
      target.require_grad = False
      output[target == 0] = 0
      loss = criterion(output, target)
      mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
      loss.backward()
      train_loss += np.sqrt(loss.data*mean_corrector)
      s += 1.
      optimizer.step()
  print('epoch: '+str(epoch)+'loss: '+ str(train_loss/s))

epoch: 1loss: tensor(1.7716)
epoch: 2loss: tensor(1.0966)
epoch: 3loss: tensor(1.0535)
epoch: 4loss: tensor(1.0383)
epoch: 5loss: tensor(1.0307)
epoch: 6loss: tensor(1.0268)
epoch: 7loss: tensor(1.0238)
epoch: 8loss: tensor(1.0220)
epoch: 9loss: tensor(1.0209)
epoch: 10loss: tensor(1.0197)
epoch: 11loss: tensor(1.0188)
epoch: 12loss: tensor(1.0186)
epoch: 13loss: tensor(1.0180)
epoch: 14loss: tensor(1.0174)
epoch: 15loss: tensor(1.0173)
epoch: 16loss: tensor(1.0169)
epoch: 17loss: tensor(1.0166)
epoch: 18loss: tensor(1.0167)
epoch: 19loss: tensor(1.0164)
epoch: 20loss: tensor(1.0163)
epoch: 21loss: tensor(1.0161)
epoch: 22loss: tensor(1.0161)
epoch: 23loss: tensor(1.0162)
epoch: 24loss: tensor(1.0160)
epoch: 25loss: tensor(1.0156)
epoch: 26loss: tensor(1.0154)
epoch: 27loss: tensor(1.0154)
epoch: 28loss: tensor(1.0150)
epoch: 29loss: tensor(1.0129)
epoch: 30loss: tensor(1.0117)
epoch: 31loss: tensor(1.0091)
epoch: 32loss: tensor(1.0090)
epoch: 33loss: tensor(1.0053)
epoch: 34loss: tens

Testing the SAE

In [23]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
  input = Variable(training_set[id_user]).unsqueeze(0)
  target = Variable(test_set[id_user]).unsqueeze(0)
  if torch.sum(target.data > 0) > 0:
    output = sae(input)
    target.require_grad = False
    output[target == 0] = 0
    loss = criterion(output, target)
    mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
    test_loss += np.sqrt(loss.data*mean_corrector)
    s += 1.
print('test loss: '+str(test_loss/s))

test loss: tensor(0.9679)
